## 1. Installing Split-Folder For Dataset Splitting

In [17]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import os
import numpy as np
import random
from tqdm.auto import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from glob import glob
import cv2
from skimage.io import imread, imshow, show

from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

seed = 42
np.random.seed = seed  # To get same random seed everytime we run the whole thing.
random.seed(seed)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Splitting Dataset Into Train And Validation Set

In [18]:
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H1")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H2")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H3")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H5")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H6")))

700
700
700
700
700


In [19]:
dataset= tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/ICCIT 2023/Selected_data')
class_names = dataset.class_names
print(class_names)

Found 3500 files belonging to 5 classes.
['H1', 'H2', 'H3', 'H5', 'H6']


In [20]:
!pip install --root-user-action=ignore split-folders
import splitfolders

In [21]:
input_folder = "/content/drive/MyDrive/ICCIT 2023/Selected_data"
output = "/content/working/"

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.90, 0.10))

Copying files: 3500 files [00:08, 407.93 files/s]


## 4. Data Augmentation & Importing Train And Validation Data

In [22]:
image_size = 256
no_class = 5
batch_size = 32
learning_rate = 0.001
num_epochs = 100

In [36]:

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=90,
                                   rescale = 1/255)

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                          rescale = 1/255)

train_generator = data_generator_with_aug.flow_from_directory(
        '/content/working/train',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        # seed = 42
        )

validation_generator = data_generator_no_aug.flow_from_directory(
        '/content/working/val',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        # seed = 42
        )

Found 3150 images belonging to 5 classes.
Found 350 images belonging to 5 classes.


## 5. Using Resnet50 As Base Model & Building the Model

In [34]:
resnet_model = ResNet50(include_top=False,weights="imagenet", input_shape=(image_size, image_size, 3))

resnet_model.trainable = False

x = resnet_model.output
x = layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dropout(0.6)(x)
x = tf.keras.layers.Dense(512,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
res = tf.keras.layers.Dense(no_class, activation="softmax")(x)

model = Model(inputs=resnet_model.input, outputs=res)

# Freezing the pre-trained layers so that we do not have to update weights ourselves
for layer in resnet_model.layers:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model.summary()

## 6. Training the Model

In [37]:
hist = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    verbose=1
)

# model.summary()

Epoch 1/30


KeyboardInterrupt: ignored

## 7. Evaluating the Model Loss and Model Accuracy

In [ ]:
acc= hist['accuracy']
val_acc= hist['val_accuracy']

loss= hist['loss']
val_loss= hist['val_loss']

plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.plot(range(30), acc, label='Training Accuracy')
plt.plot(range(30), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(30), loss, label='Training Loss')
plt.plot(range(30), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 8. Saving the Model

In [ ]:
model.save('defungi.h5')

### So, We have come to an end, Follow and Upvote If you like
### Thank You and Happy Kaggling !